In [64]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, isnull, lit, length, explode, count, upper, lower, regexp_replace, regexp_extract

In [ ]:
from pyspark.sql import SparkSession
# enableVectorizedReader is set to false to avoid issues with certain Parquet files
spark = SparkSession.builder \
    .appName("Data quality check vendas") \
    .config("spark.sql.parquet.enableVectorizedReader", "false") \
    .config("spark.hadoop.io.nativeio", "false") \
    .getOrCreate()

In [66]:
df = spark.read.parquet("/app/data/raw/vendas.parquet")

In [67]:
df.printSchema()

root
 |-- d_dt_vd: timestamp (nullable = true)
 |-- n_id_fil: long (nullable = true)
 |-- n_id_vd_fil: long (nullable = true)
 |-- v_cli_cod: string (nullable = true)
 |-- n_vlr_tot_vd: decimal(18,6) (nullable = true)
 |-- n_vlr_tot_desc: decimal(14,4) (nullable = true)
 |-- v_cpn_eml: string (nullable = true)
 |-- tp_pgt: string (nullable = true)



In [68]:
df.show(5)

+-------------------+--------+-----------+--------------------+------------+--------------+---------+-------+
|            d_dt_vd|n_id_fil|n_id_vd_fil|           v_cli_cod|n_vlr_tot_vd|n_vlr_tot_desc|v_cpn_eml| tp_pgt|
+-------------------+--------+-----------+--------------------+------------+--------------+---------+-------+
|2023-10-13 00:00:00| 2356284|34366442231|016E6FCC4F98832719BC|   55.960000|       13.9900|      NAO|A VISTA|
|2023-10-13 00:00:00| 2221184|35550863931|035D148EADC74B6C6D2F|   31.480000|       25.2100|      NAO|   NULL|
|2023-10-28 00:00:00| 2188984|37392732531|030C1011214A3317E850|    6.490000|       13.1400|      NAO|A VISTA|
|2023-10-09 00:00:00| 2608284| 3672652731|04710AFAF1FD9C48EBC3|   52.990000|       40.2700|      NAO|   NULL|
|2023-10-10 00:00:00|  238084|37059405031|028DBA5BBB05DDF47E4C|   18.540000|        8.3800|      NAO|A VISTA|
+-------------------+--------+-----------+--------------------+------------+--------------+---------+-------+
only showi

In [70]:
df.select([
    count(when(isnull(c), c)).alias(c) for c in df.columns
]).show()

+-------+--------+-----------+---------+------------+--------------+---------+-------+
|d_dt_vd|n_id_fil|n_id_vd_fil|v_cli_cod|n_vlr_tot_vd|n_vlr_tot_desc|v_cpn_eml| tp_pgt|
+-------+--------+-----------+---------+------------+--------------+---------+-------+
|      0|       0|          0|  5206154|           0|             0|        0|5268260|
+-------+--------+-----------+---------+------------+--------------+---------+-------+



In [ ]:
spark.stop()
